### 目标
在本作业中，您将练习量化经典神经网络模型以减少模型大小和延迟。此作业的目标如下：
* 了解量化的基本概念
* 实现和应用k-means量化
* 实施和应用k-means量化的量化感知训练
* 实现和应用线性量化
* 实现和应用线性量化的纯整数推理
* 从量化中获得对性能改进（例如加速）的基本了解
* 了解这些量化方法之间的差异和权衡
### 内容
有两个主要部分：K均值量化和线性量化。

总共有10个问题：
* 对于K均值量化，有3个问题（问题1-3）。
* 对于线性量化，有6个问题（问题4-9）。
* 问题10比较了k-means量化和线性量化。

### 设置
首先，安装所需的包并下载数据集和预训练模型。这里我们使用CIFAR10数据集和VGG网络，这与我们在Lab 0教程中使用的相同。

In [3]:
import os

print('Installing torchprofile...')
os.system("pip install torchprofile >nul 2>&1")

print('Installing fast-pytorch-kmeans...')
os.system("pip install fast-pytorch-kmeans >nul 2>&1")

print('All required packages have been successfully installed!')


Installing torchprofile...
Installing fast-pytorch-kmeans...
All required packages have been successfully installed!


In [4]:
import copy
import math
import random
from collections import OrderedDict, defaultdict

from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
from tqdm.auto import tqdm

import torch
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *

from torchprofile import profile_macs

assert torch.cuda.is_available(), \
"The current runtime does not have CUDA support." \
"Please go to menu bar (Runtime - Change runtime type) and select GPU"

In [5]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

In [ ]:
def download_url(url, model_dir='.', overwrite=False):
    import os, sys
    from urllib.request import urlretrieve
    target_dir = url.split('/')[-1]
    model_dir = os.path.expanduser(model_dir)
    try:
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        model_dir = os.path.join(model_dir, target_dir)
        cached_file = model_dir
        if not os.path.exists(cached_file) or overwrite:
            sys.stderr.write('Downloading: "{}" to {}\n'.format(url, cached_file))
            urlretrieve(url, cached_file)
        return cached_file
    except Exception as e:
        # remove lock file so download can be executed next time.
        os.remove(os.path.join(model_dir, 'download.lock'))
        sys.stderr.write('Failed to download from url %s' % url + '\n' + str(e) + '\n')
        return None